<a href="https://colab.research.google.com/github/Rukkya/.py/blob/main/diffusive_img_to_img.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import torch
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import os
import numpy as np
class CervicalCancerDataset(Dataset):
    def __init__(self, image_dir, transform=None):
        self.image_dir = image_dir
        if not os.path.exists(image_dir):
            raise FileNotFoundError(f"The directory '{image_dir}' does not exist. Please provide a valid path.")
        self.image_names = os.listdir(image_dir)
        self.transform = transform

    def __len__(self):
        return len(self.image_names)

    def __getitem__(self, idx):
        img_name = os.path.join(self.image_dir, self.image_names[idx])
        image = Image.open(img_name).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize to [-1, 1]
])

image_dir = '/root/.cache/kagglehub/datasets/prahladmehandiratta/cervical-cancer-largest-dataset-sipakmed/versions/1'
dataset = CervicalCancerDataset(image_dir, transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [4]:
import kagglehub

path = kagglehub.dataset_download("prahladmehandiratta/cervical-cancer-largest-dataset-sipakmed")

print("Path to dataset files:", path)

100%|██████████| 6.40G/6.40G [01:36<00:00, 71.3MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/prahladmehandiratta/cervical-cancer-largest-dataset-sipakmed/versions/1


In [8]:
def add_noise(x0, t, beta_start=0.0001, beta_end=0.02, T=1000):

    beta = torch.linspace(beta_start, beta_end, T)
    alpha = 1 - beta
    alpha_hat = torch.cumprod(alpha, dim=0)

    noise = torch.randn_like(x0)
    noisy_image = torch.sqrt(alpha_hat[t])[:, None, None] * x0 + torch.sqrt(1 - alpha_hat[t])[:, None, None] * noise
    return noisy_image, noise


In [18]:
import torch.nn as nn

class ConditionalUNet(nn.Module):
    def __init__(self):
        super(ConditionalUNet, self).__init__()
        self.enc1 = nn.Sequential(nn.Conv2d(6, 64, kernel_size=3, padding=1), nn.ReLU())
        self.enc2 = nn.Sequential(nn.Conv2d(64, 128, kernel_size=3, padding=1), nn.ReLU(), nn.MaxPool2d(2))
        self.bottleneck = nn.Sequential(nn.Conv2d(128, 256, kernel_size=3, padding=1), nn.ReLU())
        self.upconv1 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.dec1 = nn.Sequential(nn.Conv2d(128 + 128, 128, kernel_size=3, padding=1), nn.ReLU())
        self.upconv2 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.dec2 = nn.Sequential(nn.Conv2d(64 + 64, 64, kernel_size=3, padding=1), nn.ReLU())
        self.out_conv = nn.Conv2d(64, 3, kernel_size=3, padding=1)

    def forward(self, x, y):
        x1 = self.enc1(x)
        x2 = self.enc2(x1)
        x3 = self.bottleneck(x2)
        x4 = self.upconv1(x3)
        x4 = self.dec1(torch.cat([x4, y], dim=1))
        x5 = self.upconv2(x4)
        x5 = self.dec2(torch.cat([x5, y], dim=1))
        out = self.out_conv(x5)
        return out


In [19]:
import torch.optim as optim

model = ConditionalUNet().to('cpu')
optimizer = optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.MSELoss()

def train(model, dataloader, optimizer, criterion, epochs=10, T=1000):
    for epoch in range(epochs):
        for x0 in dataloader:
            x0 = x0.to(device)
            t = torch.randint(0, T, (x0.size(0),), device=x0.device)
            noisy_images, noise = add_noise(x0, t)
            pred_noise = model(noisy_images, x0)
            loss = criterion(pred_noise, noise)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item()}")

train(model, dataloader, optimizer, criterion)


IsADirectoryError: [Errno 21] Is a directory: '/root/.cache/kagglehub/datasets/prahladmehandiratta/cervical-cancer-largest-dataset-sipakmed/versions/1/im_Metaplastic'

In [20]:
def generate_image(model, noise, T=1000, device='cuda'):
    model.eval()
    with torch.no_grad():
        for t in reversed(range(T)):
            noisy_image = noise
            predicted_noise = model(noisy_image, noisy_image)
            noise = predicted_noise
        return noisy_image

noise = torch.randn(1, 3, 128, 128).to('cuda')
generated_image = generate_image(model, noise)

generated_image = generated_image.squeeze().cpu().detach().numpy().transpose(1, 2, 0)
generated_image = (generated_image * 0.5 + 0.5) * 255
generated_image = Image.fromarray(generated_image.astype(np.uint8))
generated_image.show()


NameError: name 'device' is not defined